In [1]:
%matplotlib inline 
#%load_ext autoreload 
%reload_ext autoreload
%autoreload 2
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

from __future__ import (division, 
                        print_function)

import os
import sys
import copy

import h5py
import numpy as np
import scipy
from scipy.interpolate import interp1d

from astropy.io import fits
from astropy import units as u
from astropy.stats import sigma_clip
from astropy.table import Table, Column
from astropy.utils.console import ProgressBar
from astropy.convolution import convolve, Box1DKernel


import matplotlib as mpl
import matplotlib.mlab as ml
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from matplotlib.patches import Ellipse
from matplotlib.ticker import NullFormatter, MaxNLocator, FormatStrFormatter
from matplotlib.collections import PatchCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
tickFormat = FormatStrFormatter('$\mathbf{%g}$') 
mpl.rcParams.update({'xtick.color': 'k'})
mpl.rcParams.update({'ytick.color': 'k'})
mpl.rcParams.update({'font.size': 20})

import cosmology
c=cosmology.Cosmo(H0=70.0, omega_m=0.3, omega_l=0.7, flat=1)


import sys
sys.path.append('/Users/RAJ/github/kungpao')
from kungpao.galsbp import galSBP
from kungpao.display import display_single, random_cmap
import statsmodels.api as sm
from scipy.interpolate import interp1d 
from kungpao import io
from kungpao import utils
from kungpao import detection
from kungpao import imtools

import sep
from palettable.colorbrewer.sequential import Greys_9, OrRd_9, Blues_9, Purples_9, YlGn_9
BLK = Greys_9.mpl_colormap
ORG = OrRd_9.mpl_colormap
BLU = Blues_9.mpl_colormap
GRN = YlGn_9.mpl_colormap

/Users/RAJ/anaconda/envs/illustris_profiles/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/RAJ/anaconda/envs/illustris_profiles/lib/python2.7/site-packages/pyraf/irafimport.py:116: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  retval = _originalImport(name, globals, locals, fromlist, level)


In [2]:
x_images = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf/bin.macosx/x_images.e'
TBL = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf_extern/tables/bin.macosx/x_ttools.e'
ISO = '/Users/RAJ/anaconda/envs/illustris_profiles/iraf_extern/stsdas/bin.macosx/x_isophote.e'

In [3]:
kernel3 = np.asarray([[0.092163, 0.221178, 0.296069, 0.221178, 0.092163],
                      [0.221178, 0.530797, 0.710525, 0.530797, 0.221178],
                      [0.296069, 0.710525, 0.951108, 0.710525, 0.296069],
                      [0.221178, 0.530797, 0.710525, 0.530797, 0.221178],
                      [0.092163, 0.221178, 0.296069, 0.221178, 0.092163]])

In [4]:
def get_pixel_scale(file):
    f = h5py.File(file, 'r')
    map_size = f['config'].attrs['map_range_min']
    n_pixels = f['config'].attrs['map_npixel']
    pixel_scale=2 * (map_size/n_pixels)

    return pixel_scale

In [ ]:
def get_ngalaxies(file):
    f = h5py.File(file, 'r')
    n_galaxies = len(f['catsh_id'])
    return n_galaxies

In [1]:
illustris_quick_file_hdf5='/Users/RAJ/summer_work_2018/notebook_tng/quick/galaxies_tng100_072_agez_lowres.hdf5'

In [6]:
pixel_scale_quick = get_pixel_scale(illustris_quick_file_hdf5)

In [ ]:
n_galaxies = get_ngalaxies(illustris_quick_file_hdf5)

In [8]:
illustris_file = h5py.File(illustris_quick_file_hdf5, 'r')

In [9]:
illustris_direc = '/Users/RAJ/summer_work_2018/notebook_tng/quick/ellipse' 

In [8]:
def save_to_fits(image, name):
    """
    Save a 2-D array as fits image.
    """
    hdu = fits.PrimaryHDU(image)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(name, overwrite=True)

    return

In [12]:
def get_mass_maps(sim_file,direc):

    # Load general simulation and galaxy properties
    f = h5py.File(sim_file, 'r')
    n_galaxies = len(f['catsh_id'])
    cat_sh_mstar = np.array(f['catsh_SubhaloMassType'][:, 4])

    cen_insitu = np.array(f['map_star_rho_insitu_xy'])
    cen_exsitu = np.array(f['map_star_rho_exsitu_xy'])
    map_stars_cen = cen_exsitu + cen_insitu



    map_size = f['config'].attrs['map_range_min']
    n_pixels = f['config'].attrs['map_npixel']
    pixel_scale=2 * (map_size/n_pixels)
    illustris_maps = []

    f.close()
    
    
    for gal_n in range(n_galaxies):
        #make maps
        img_ins = cen_insitu[gal_n]
        img_exs = cen_exsitu[gal_n]
        img_cen = map_stars_cen[gal_n] * (pixel_scale ** 2) # Central
        
        #catalog mass
        m_cat = np.log10(cat_sh_mstar[gal_n])
        prefix = 'illustris_%s' % str(gal_n).strip()
        cen_fits = os.path.join(direc, prefix + "_cen.fits" )
        _ = io.save_to_fits(img_cen, cen_fits)
        # In-situ stars
        ins_fits = os.path.join(direc, prefix + "_ins.fits" )
        _ = io.save_to_fits(img_ins, ins_fits)

        # Ex-situ stars
        exs_fits = os.path.join(direc, prefix + "_exs.fits" )
        _ = io.save_to_fits(img_exs, exs_fits)

        illustris_maps.append(np.stack([img_cen, img_ins, img_exs]))


    return illustris_maps

In [13]:
illustris_maps = get_mass_maps(illustris_quick_file_hdf5, illustris_direc) #making the mass maps and it is being stored in illustris_direc

In [14]:
np.save(os.path.join(illustris_direc, 'illustris_quick_maps.npy'), np.asarray(illustris_maps))

In [17]:
def illustris_aperture_photometry(img, img_cen_x=None, img_cen_y=None, scale=pixel_scale_quick):
    """Aperture photometry information for illustris galaxy."""
    bkg = sep.Background(img, bw=10, bh=10, fw=5, fh=5)
    img_sub = img - bkg
    objects = sep.extract(img_sub, 20.0, filter_kernel=kernel3)
    
    img_h, img_w = img_sub.shape
    if img_cen_x is None:
        img_cen_x = img_w / 2.0
    if img_cen_y is None:
        img_cen_y = img_h / 2.0

    # Find the object at the center
    index = np.argmin(np.sqrt((objects['x'] - img_cen_x) ** 2.0 + 
                              (objects['y'] - img_cen_y) ** 2.0))

    # Get the naive ba, theta, xcen, ycen
    ba = objects[index]['b'] / objects[index]['a']
    theta = objects[index]['theta']
    xcen, ycen = objects[index]['x'], objects[index]['y']
    
    # Mass within different apertures
    rad = np.asarray([10.0, 20.0, 30.0, 40.0, 50.0, 100.0, 150.0, 200.0])
    maper = sep.sum_ellipse(img, xcen, ycen, rad / scale, rad /scale * ba, theta, 1.0, 
                            bkgann=None, subpix=11)[0]

    aper_results = {'x': xcen, 'y': ycen, 'ba': ba, 'pa': theta,
                    'rad': rad, 'maper': np.log10(maper)}
    
    return aper_results

In [18]:
illustris_0_ins_aper = [illustris_aperture_photometry(gal[1]) for gal in illustris_maps]
illustris_0_exs_aper = [illustris_aperture_photometry(gal[2]) for gal in illustris_maps]
illustris_0_cen_aper = [illustris_aperture_photometry(gal[0]) for gal in illustris_maps]

In [21]:
np.save(os.path.join(illustris_direc, 'illustris_quick_aper_ins.npy'), np.asarray(illustris_0_ins_aper))
np.save(os.path.join(illustris_direc, 'illustris_quick_aper_exs.npy'), np.asarray(illustris_0_exs_aper))
np.save(os.path.join(illustris_direc, 'illustris_quick_aper_cen.npy'), np.asarray(illustris_0_cen_aper))

In [38]:
def get_fits_name(idx, img_type='cen', illustris_dir=illustris_direc):
    """Return the name of the FITS file."""
    prefix = 'illustris_%s' % str(idx).strip()
    suffix = '_%s.fits' % (img_type)
    
    return os.path.join(illustris_dir, prefix + suffix)

def illustris_get_1d_prof(fits, aper_result, isophote=ISO, xttools=TBL,
                    pixel_scale=pixel_scale_quick):
    """Get Step 2 and Step 3 1-D profile."""
    xcen, ycen = aper_result['x'], aper_result['y']
    ba = aper_result['ba'] 
    pa = utils.normalize_angle(aper_result['pa'] * 180.0 / np.pi + 90.0,
                               lower=-90, upper=90, b=True)
    
    # Step 2 to get ellipticity and position angle profiles
    
    ell_2, bin_2 = galSBP.galSBP(fits, galX=xcen, galY=ycen, 
                                 maxSma=100, iniSma=8.0, 
                                 verbose=False, savePng=False, 
                                 saveOut=True, expTime=1.0, 
                                 pix=pixel_scale, zpPhoto=0.0,
                                 galQ=ba, galPA=pa, 
                                 stage=2, minSma=0.0, 
                                 ellipStep=0.15,
                                 isophote=isophote, 
                                 xttools=xttools, 
                                 uppClip=2.5, lowClip=3.0, 
                                 maxTry=9, nClip=2, intMode='mean', 
                                 updateIntens=False)
    
    
    # Step 3 to get mass density profiles
    ell_3, bin_3 = galSBP.galSBP(fits, galX=xcen, galY=ycen, 
                                 maxSma=130, iniSma=8.0, 
                                 verbose=False, savePng=False, 
                                 saveOut=True, expTime=1.0, 
                                 pix=pixel_scale, zpPhoto=0.0,
                                 galQ=ba, galPA=pa, 
                                 stage=3, minSma=0.0, 
                                 ellipStep=0.08,
                                 isophote=isophote, 
                                 xttools=xttools, 
                                 uppClip=2.5, lowClip=3.0, 
                                 maxTry=2, nClip=2, intMode='mean', 
                                 updateIntens=False)
    
    return ell_2, ell_3, bin_2, bin_3
    
    
def illustris_get_1d_force(fits, binary, aper_result, isophote=ISO, xttools=TBL,
                     pixel_scale= pixel_scale_quick):
    """Get Step 4 force photometry 1-D profile."""
    xcen, ycen = aper_result['x'], aper_result['y']
    ba = aper_result['ba'] 
    pa = utils.normalize_angle(aper_result['pa'] * 180.0 / np.pi + 90.0,
                               lower=-90, upper=90, b=True)
    
    # Step 2 to get ellipticity and position angle profiles
    ell_4, bin_4 = galSBP.galSBP(fits, inEllip=binary, 
                                 galX=xcen, galY=ycen, 
                                 verbose=False, savePng=False, 
                                 saveOut=True, expTime=1.0, 
                                 pix=pixel_scale, zpPhoto=0.0,
                                 stage=4, isophote=isophote, xttools=xttools, 
                                 uppClip=2.5, lowClip=3.0, 
                                 maxTry=2, nClip=2, intMode='mean', 
                                 updateIntens=False)
    
    return ell_4, bin_4


def illustris_get_1d_all(idx, aper_cen, aper_ins, aper_exs, 
                         illustris_dir=illustris_direc):
    """Get all the necessary 1-D profiles for single illustris galaxy."""
    print("# Dealing with galaxy %d" % idx)
    
    # Get the FITS files
    fits_cen = get_fits_name(idx, img_type='cen', illustris_dir=illustris_direc)
    fits_ins = get_fits_name(idx, img_type='ins', illustris_dir=illustris_direc)
    fits_exs = get_fits_name(idx, img_type='exs', illustris_dir=illustris_direc)

    # Step 2, Step 3 for central galaxy
    ell_cen = illustris_get_1d_prof(fits_cen, aper_cen)
    ell_cen_2, ell_cen_3, bin_cen_2, bin_cen_3 = ell_cen

    # Step 2, Step 3 for In-situ component
    ell_ins = illustris_get_1d_prof(fits_ins, aper_ins)
    ell_ins_2, ell_ins_3, bin_ins_2, bin_ins_3 = ell_ins

    # Step 2, Step 3 for Ex-situ component
    ell_exs = illustris_get_1d_prof(fits_exs, aper_exs)
    ell_exs_2, ell_exs_3, bin_exs_2, bin_exs_3 = ell_exs

    # Step 4 force 1-D profiles for in-situ and ex-situ components
    ell_ins_4, bin_ins_4 = illustris_get_1d_force(fits_ins, bin_cen_3, aper_cen)
    ell_exs_4, bin_exs_4 = illustris_get_1d_force(fits_exs, bin_cen_3, aper_cen)
    
    return {'ell_cen_2': ell_cen_2, 'ell_cen_3': ell_cen_3,
            'ell_ins_2': ell_ins_2, 'ell_ins_3': ell_ins_3,
            'ell_exs_2': ell_exs_2, 'ell_exs_3': ell_exs_3,
            'ell_ins_4': ell_ins_4, 'ell_exs_4': ell_exs_4,
            'bin_cen_3': bin_cen_3}

In [44]:
illustris_0_ell = [illustris_get_1d_all(idx,
                            illustris_0_cen_aper[idx], 
                            illustris_0_ins_aper[idx], 
                            illustris_0_exs_aper[idx] 
                             ) for idx in np.arange(n_galaxies)] 

# Dealing with galaxy 0
# Dealing with galaxy 1


/Users/RAJ/github/kungpao/kungpao/galsbp/galSBP.py:579: RuntimeWarning: divide by zero encountered in log10
  sbpOri = zp - 2.5 * np.log10(intensOri / (pixArea * exptime))
/Users/RAJ/github/kungpao/kungpao/galsbp/galSBP.py:580: RuntimeWarning: divide by zero encountered in log10
  sbpSub = zp - 2.5 * np.log10(intensSub / (pixArea * exptime))
/Users/RAJ/github/kungpao/kungpao/galsbp/galSBP.py:590: RuntimeWarning: divide by zero encountered in log10
  (pixArea * exptime))
/Users/RAJ/github/kungpao/kungpao/galsbp/galSBP.py:591: RuntimeWarning: invalid value encountered in subtract
  sbp_err = (sbpSub - sbp_low)
/Users/RAJ/github/kungpao/kungpao/galsbp/galSBP.py:1617: RuntimeWarning: divide by zero encountered in log10
  expTime))


# Dealing with galaxy 2
# Dealing with galaxy 3
# Dealing with galaxy 4
# Dealing with galaxy 5
# Dealing with galaxy 6
# Dealing with galaxy 7
# Dealing with galaxy 8
# Dealing with galaxy 9
# Dealing with galaxy 10
# Dealing with galaxy 11
# Dealing with galaxy 12
# Dealing with galaxy 13
# Dealing with galaxy 14
# Dealing with galaxy 15
# Dealing with galaxy 16
# Dealing with galaxy 17
# Dealing with galaxy 18
# Dealing with galaxy 19
# Dealing with galaxy 20
# Dealing with galaxy 21
# Dealing with galaxy 22
# Dealing with galaxy 23
# Dealing with galaxy 24
# Dealing with galaxy 25
# Dealing with galaxy 26
# Dealing with galaxy 27
# Dealing with galaxy 28
# Dealing with galaxy 29
# Dealing with galaxy 30
# Dealing with galaxy 31
# Dealing with galaxy 32
# Dealing with galaxy 33
# Dealing with galaxy 34
# Dealing with galaxy 35
# Dealing with galaxy 36
# Dealing with galaxy 37
# Dealing with galaxy 38
# Dealing with galaxy 39
# Dealing with galaxy 40
# Dealing with galaxy 41
# Dealin

In [46]:
np.save(os.path.join(illustris_direc, 'illustris_0_ell_quick.npy'), np.asarray(illustris_0_ell))